In [5]:
import os
import h5py
import numpy as np
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

# Import the helper function from your helpers module.
from helpers import process_image_full
from update_h5 import create_updated_h5

# Define your image and mask file paths.
image_file = "/home/bubl3932/files/UOX1/UOX1_original/UOX1_sub_rb/UOX1_sub_radial_backgrounds.h5"
mask_file = "/home/bubl3932/mask/pxmask.h5"

# Define processing parameters.
threshold = 0.1
max_iters = 10
step_size = 1
n_steps = 5
n_wedges = 4
n_rad_bins = 100
plot_profiles = False
chunk_size = 100  # Adjust based on available memory

csv_file = os.path.join(os.path.dirname(image_file), "centers.csv")

# Remove existing CSV file if present.
if os.path.exists(csv_file):
    os.remove(csv_file)
header_written = False

def process_chunk(start_idx, images_chunk, mask, pbar):
    # Build list of argument tuples, one per image.
    args = []
    for idx, img in enumerate(images_chunk):
        frame_number = start_idx + idx
        # Each tuple contains all parameters required by process_image:
        args.append((
            frame_number,
            img,
            mask,
            threshold,
            max_iters,
            step_size,
            n_steps,
            n_wedges,
            n_rad_bins,
            plot_profiles
        ))
        
    results = []
    with Pool() as pool:
        # Use imap_unordered to yield each result as soon as it is ready.
        for result in pool.imap_unordered(process_image_full, args):
            results.append(result)
            pbar.update(1)  # Update the progress bar by one image.
    return results

if __name__ == '__main__':
    with h5py.File(image_file, 'r') as f_img, h5py.File(mask_file, 'r') as f_mask:
        dataset_images = f_img['/entry/data/images']
        mask = f_mask['/mask'][:].astype(bool)
        # mask = np.ones(dataset_images[0].shape, dtype=bool) # Uncomment to use a mask of all True.



        n_images = dataset_images.shape[0]
        print(f"Total images: {n_images}")

        all_results = []
        # Create a tqdm progress bar with total = number of images.
        with tqdm(total=n_images, desc="Processing centers") as pbar:
            for i in range(0, n_images, chunk_size):
                images_chunk = dataset_images[i:i+chunk_size].astype(np.float32)
                chunk_results = process_chunk(i, images_chunk, mask, pbar)
                # Sort the chunk's results by frame_number.
                chunk_results = sorted(chunk_results, key=lambda x: x[0])
                all_results.extend(chunk_results)

                # Create a DataFrame for the chunk.
                df_chunk = pd.DataFrame(chunk_results, columns=["frame_number", "center"])
                swapped_centers = [(center[1], center[0]) for center in df_chunk['center']]
                df_chunk[['center_x', 'center_y']] = pd.DataFrame(swapped_centers, index=df_chunk.index)
                df_chunk = df_chunk.drop(columns=["center"])

                # Write/append to CSV.
                mode = "w" if not header_written else "a"
                df_chunk.to_csv(csv_file, index=False, mode=mode, header=(not header_written))
                header_written = True

        print("Finished processing and saved centers to", csv_file)
        
        new_h5_path = os.path.join(os.path.dirname(image_file),os.path.splitext(image_file)[0] + "_ICF.h5")

        # Run the function to create the updated HDF5 file
        create_updated_h5(image_file, new_h5_path, csv_file)


Total images: 100


Processing centers:   0%|          | 0/100 [00:00<?, ?it/s]

Processing centers: 100%|██████████| 100/100 [02:42<00:00,  1.62s/it]


Finished processing and saved centers to /home/bubl3932/files/UOX1/UOX1_original/UOX1_sub_rb/centers.csv
New HDF5 file created: /home/bubl3932/files/UOX1/UOX1_original/UOX1_sub_rb/UOX1_sub_radial_backgrounds_ICF.h5
Center coordinates and detector shifts have been updated.


In [ ]:

# This cell imports the `create_updated_h5` function from your saved `update_h5.py` module and calls it using your file paths.
from update_h5 import create_updated_h5

new_h5_path = os.path.join(os.path.dirname(image_file),os.path.splitext(image_file)[0] + "_ICF.h5")

# Run the function to create the updated HDF5 file
create_updated_h5(image_file, new_h5_path, csv_file)
